In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("Step2-WarmUp") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.executor.memory", "500mb") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/01 23:11:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
! ls -lh /opt/spark-notebooks

total 16K
drwxrwxrwx 1 root root 4.0K Jan  1 18:37 'Step1-Warming Up'
drwxrwxrwx 1 root root 4.0K Jan  1 18:37  Step2-Kafka
drwxrwxrwx 1 root root 4.0K Jan  1 18:37  Step3-Designing-Data-Pipeline
-rwxrwxrwx 1 root root  15K Jan  1 23:10  step2-warm-up.ipynb


In [4]:
! ls -lh /opt/spark-data

total 4.0M
-rwxrwxrwx 1 root root   12K Aug 19 18:46 Case.csv
-rwxrwxrwx 1 root root  1.6M Sep 12 08:11 MTA-Bus-Time_.2014-08-01.txt
-rwxrwxrwx 1 root root   19K Aug 19 18:46 Region.csv
-rwxrwxrwx 1 root root   90K Aug 19 18:46 TimeProvince.csv
drwxrwxrwx 1 root root  4.0K Jan  1 18:37 datasets
-rwxrwxrwx 1 root root  1.3M Sep 11 16:59 products.parquet
-rwxrwxrwx 1 root root 1023K Sep 11 16:59 sales.parquet
-rwxrwxrwx 1 root root  4.7K Sep 11 16:59 sellers.parquet
-rwxrwxrwx 1 root root  1.2K Oct  4 12:59 user.json


In [5]:
products_table = spark.read.parquet("/opt/spark-data/products.parquet")
sales_table = spark.read.parquet("/opt/spark-data/sales.parquet")
sellers_table = spark.read.parquet("/opt/spark-data/sellers.parquet")

In [6]:
products_table.show(10)

+----------+--------------+-----+
|product_id|  product_name|price|
+----------+--------------+-----+
|         1|product_000001|   30|
|         2|product_000002|   91|
|         3|product_000003|   37|
|         4|product_000004|  145|
|         5|product_000005|  128|
|         6|product_000006|   66|
|         7|product_000007|  145|
|         8|product_000008|   51|
|         9|product_000009|   44|
|        10|product_000010|   53|
+----------+--------------+-----+
only showing top 10 rows



In [7]:
sellers_table.show(10)

+---------+-----------+------------+
|seller_id|seller_name|daily_target|
+---------+-----------+------------+
|        0|   seller_0|     2500000|
|        1|   seller_1|       33202|
|        2|   seller_2|     1108933|
|        3|   seller_3|     1974673|
|        4|   seller_4|     1097088|
|        5|   seller_5|      988071|
|        6|   seller_6|     1061493|
|        7|   seller_7|      347565|
|        8|   seller_8|     1634449|
|        9|   seller_9|     1433258|
+---------+-----------+------------+
only showing top 10 rows



In [8]:
sales_table.show(10)

+--------+----------+---------+----------+---------------+--------------------+
|order_id|product_id|seller_id|      date|num_pieces_sold|       bill_raw_text|
+--------+----------+---------+----------+---------------+--------------------+
|       1|     57164|       69|2021-10-04|             20|tvuerrnvlBwhejtyz...|
|       2|     17466|        4|2021-10-03|             80|ezoexpsddxnahgkxr...|
|       3|     25747|       81|2021-10-08|             24|rrzavxtphkxhyiicu...|
|       4|     53745|       59|2021-10-02|             92|jklcqmgjbfqxtsugt...|
|       5|     31885|        5|2021-10-06|             10|azdujdzlhddjebywm...|
|       6|     36014|       52|2021-10-08|             71|jpjwwfckwolwphpkq...|
|       7|      1709|       59|2021-10-05|              3|gcjrymmjlcmkhoyxq...|
|       8|      1736|       98|2021-10-04|             87|ukhygpejttpbwwden...|
|       9|     58029|       32|2021-10-04|             10|uzfdkgybsrraxpspc...|
|      10|     46330|       44|2021-10-0

In [9]:
products_table.describe().show()

+-------+-----------------+--------------+-----------------+
|summary|       product_id|  product_name|            price|
+-------+-----------------+--------------+-----------------+
|  count|           100000|        100000|           100000|
|   mean|          50000.5|          null|         75.43343|
| stddev|28867.65779668774|          null|43.41501971754077|
|    min|                1|product_000001|                1|
|    max|           100000|product_100000|              150|
+-------+-----------------+--------------+-----------------+



In [10]:
sales_table.printSchema()

root
 |-- order_id: long (nullable = true)
 |-- product_id: long (nullable = true)
 |-- seller_id: long (nullable = true)
 |-- date: string (nullable = true)
 |-- num_pieces_sold: long (nullable = true)
 |-- bill_raw_text: string (nullable = true)



In [11]:
#   Print the number of orders
print("Number of Orders: {}".format(sales_table.count()))

#   Print the number of sellers
print("Number of sellers: {}".format(sellers_table.count()))

#   Print the number of products
print("Number of products: {}".format(products_table.count()))

Number of Orders: 2000
Number of sellers: 101
Number of products: 100000


In [12]:
products_table.cache()
sales_table.cache()
sellers_table.cache()

DataFrame[seller_id: bigint, seller_name: string, daily_target: bigint]

In [13]:
products_table.select(products_table.product_id).show(5)

+----------+
|product_id|
+----------+
|         1|
|         2|
|         3|
|         4|
|         5|
+----------+
only showing top 5 rows



In [14]:
products_table.select(col('product_id')).show()

+----------+
|product_id|
+----------+
|         1|
|         2|
|         3|
|         4|
|         5|
|         6|
|         7|
|         8|
|         9|
|        10|
|        11|
|        12|
|        13|
|        14|
|        15|
|        16|
|        17|
|        18|
|        19|
|        20|
+----------+
only showing top 20 rows



In [15]:
#   Output how many products have been actually sold at least once
print("Number of products sold at least once")
sales_table.agg(countDistinct(col("product_id"))).show()


Number of products sold at least once
+-----------------+
|count(product_id)|
+-----------------+
|             1981|
+-----------------+



In [16]:
#   Output which is the product that has been sold in more orders
print("Product present in more orders")
sales_table.groupBy(col("product_id")).agg(
    count("*").alias("cnt")).orderBy(col("cnt").desc()).limit(10).show()

Product present in more orders
+----------+---+
|product_id|cnt|
+----------+---+
|     99115|  2|
|     33962|  2|
|     17921|  2|
|     86810|  2|
|     25341|  2|
|     36878|  2|
|     52666|  2|
|     36569|  2|
|     51975|  2|
|     31673|  2|
+----------+---+



In [17]:
#   Output which is the product that has been sold in more orders
print("Product present in more orders")
sales_table.groupBy(col("product_id")).agg(
    count("*").alias("cnt")).orderBy(col("cnt").desc()).limit(10).explain()

Product present in more orders
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=10, orderBy=[cnt#854L DESC NULLS LAST], output=[product_id#7L,cnt#854L])
   +- HashAggregate(keys=[product_id#7L], functions=[count(1)])
      +- Exchange hashpartitioning(product_id#7L, 200), ENSURE_REQUIREMENTS, [id=#386]
         +- HashAggregate(keys=[product_id#7L], functions=[partial_count(1)])
            +- InMemoryTableScan [product_id#7L]
                  +- InMemoryRelation [order_id#6L, product_id#7L, seller_id#8L, date#9, num_pieces_sold#10L, bill_raw_text#11], StorageLevel(disk, memory, deserialized, 1 replicas)
                        +- *(1) ColumnarToRow
                           +- FileScan parquet [order_id#6L,product_id#7L,seller_id#8L,date#9,num_pieces_sold#10L,bill_raw_text#11] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/opt/spark-data/sales.parquet], PartitionFilters: [], PushedFilters: [], ReadSch

In [18]:
products_table.createOrReplaceTempView("products")
sales_table.createOrReplaceTempView("sales")
sellers_table.createOrReplaceTempView("sellers")

In [19]:
spark.sql("select count(distinct product_id) as `product sold`  from sales").show()


+------------+
|product sold|
+------------+
|        1981|
+------------+



In [20]:
print("Product present in more orders")

spark.sql("""select product_id, count(*) as cnt
            from sales
            group by 1
            order by 2 desc 
            limit 10""").show()

Product present in more orders
+----------+---+
|product_id|cnt|
+----------+---+
|     99115|  2|
|     33962|  2|
|     17921|  2|
|     86810|  2|
|     25341|  2|
|     36878|  2|
|     52666|  2|
|     36569|  2|
|     51975|  2|
|     31673|  2|
+----------+---+



In [21]:
print("Product present in more orders")

df10=spark.sql("""select product_name, count(*) as cnt
            from sales s inner join products p on p.product_id = s.product_id
            group by 1 
            having cnt >1
            order by 2 desc 
            """)
df10.show()

Product present in more orders


+--------------+---+
|  product_name|cnt|
+--------------+---+
|product_059515|  2|
|product_051975|  2|
|product_048571|  2|
|product_036878|  2|
|product_015145|  2|
|product_098784|  2|
|product_036569|  2|
|product_052666|  2|
|product_063976|  2|
|product_038144|  2|
|product_031673|  2|
|product_070540|  2|
|product_099115|  2|
|product_010594|  2|
|product_033962|  2|
|product_086810|  2|
|product_017921|  2|
|product_025341|  2|
|product_001709|  2|
+--------------+---+



In [22]:
df10.count()

19

In [23]:
df20=spark.sql("""select product_name, count(*)
            from sales s right outer join products p on p.product_id = s.product_id
            group by 1 
            """)
# print(df20.count())
print(df20.show(1000))

+--------------+--------+
|  product_name|count(1)|
+--------------+--------+
|product_000227|       1|
|product_000328|       1|
|product_000530|       1|
|product_000718|       1|
|product_000900|       1|
|product_001021|       1|
|product_001070|       1|
|product_001172|       1|
|product_001362|       1|
|product_001368|       1|
|product_001724|       1|
|product_001796|       1|
|product_002124|       1|
|product_002257|       1|
|product_002386|       1|
|product_002574|       1|
|product_002693|       1|
|product_003396|       1|
|product_003571|       1|
|product_003634|       1|
|product_003645|       1|
|product_003780|       1|
|product_004129|       1|
|product_004259|       1|
|product_004682|       1|
|product_004967|       1|
|product_005046|       1|
|product_005053|       1|
|product_005099|       1|
|product_005820|       1|
|product_005864|       1|
|product_006357|       1|
|product_006604|       1|
|product_007078|       1|
|product_007163|       1|
|product_007

In [24]:
print("Product present in more orders")

df20=spark.sql("""select product_name, count(*)
            from sales s right outer join products p on p.product_id = s.product_id
            group by 1 
            order by 2 desc 
            """)
df20.cache()
print(df20.count())
print(df20.show())

Product present in more orders


100000
+--------------+--------+
|  product_name|count(1)|
+--------------+--------+
|product_059515|       2|
|product_051975|       2|
|product_048571|       2|
|product_036878|       2|
|product_015145|       2|
|product_098784|       2|
|product_036569|       2|
|product_052666|       2|
|product_063976|       2|
|product_038144|       2|
|product_031673|       2|
|product_070540|       2|
|product_099115|       2|
|product_010594|       2|
|product_033962|       2|
|product_086810|       2|
|product_017921|       2|
|product_025341|       2|
|product_001709|       2|
|product_009458|       1|
+--------------+--------+
only showing top 20 rows

None


In [25]:
print(df20.count())
print(df20.show())

100000
+--------------+--------+
|  product_name|count(1)|
+--------------+--------+
|product_059515|       2|
|product_051975|       2|
|product_048571|       2|
|product_036878|       2|
|product_015145|       2|
|product_098784|       2|
|product_036569|       2|
|product_052666|       2|
|product_063976|       2|
|product_038144|       2|
|product_031673|       2|
|product_070540|       2|
|product_099115|       2|
|product_010594|       2|
|product_033962|       2|
|product_086810|       2|
|product_017921|       2|
|product_025341|       2|
|product_001709|       2|
|product_009458|       1|
+--------------+--------+
only showing top 20 rows

None


In [26]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold","20m")

In [27]:

df30=spark.sql("""select product_name, count(*)
            from sales s join products p on p.product_id = s.product_id
            group by 1 
            order by 2 asc 
            """)
df30.show()

+--------------+--------+
|  product_name|count(1)|
+--------------+--------+
|product_006357|       1|
|product_015968|       1|
|product_038272|       1|
|product_042787|       1|
|product_043227|       1|
|product_044786|       1|
|product_053513|       1|
|product_074236|       1|
|product_081288|       1|
|product_088450|       1|
|product_016227|       1|
|product_020277|       1|
|product_031153|       1|
|product_045124|       1|
|product_050309|       1|
|product_064223|       1|
|product_085683|       1|
|product_007865|       1|
|product_013799|       1|
|product_034593|       1|
+--------------+--------+
only showing top 20 rows



In [28]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")
df40=spark.sql("""select product_name, count(*)
            from sales s join products p on p.product_id = s.product_id
            group by 1 
            order by 2 asc 
            """)
df40.show()

+--------------+--------+
|  product_name|count(1)|
+--------------+--------+
|product_006357|       1|
|product_015968|       1|
|product_038272|       1|
|product_042787|       1|
|product_043227|       1|
|product_044786|       1|
|product_053513|       1|
|product_074236|       1|
|product_081288|       1|
|product_088450|       1|
|product_016227|       1|
|product_020277|       1|
|product_031153|       1|
|product_045124|       1|
|product_050309|       1|
|product_064223|       1|
|product_085683|       1|
|product_007865|       1|
|product_013799|       1|
|product_034593|       1|
+--------------+--------+
only showing top 20 rows



In [29]:
# SELECT /*+  BROADCASTJOIN(small) */ * 

df50=spark.sql("""select /*+  BROADCASTJOIN(s) */   product_name, count(*)  
            from products p join sales s on p.product_id = s.product_id
            group by 1 
            order by 2 asc 
            """)
print(df50.count())
df50.show()

1981
+--------------+--------+
|  product_name|count(1)|
+--------------+--------+
|product_006357|       1|
|product_015968|       1|
|product_038272|       1|
|product_042787|       1|
|product_043227|       1|
|product_044786|       1|
|product_053513|       1|
|product_074236|       1|
|product_081288|       1|
|product_088450|       1|
|product_016227|       1|
|product_020277|       1|
|product_031153|       1|
|product_045124|       1|
|product_050309|       1|
|product_064223|       1|
|product_085683|       1|
|product_007865|       1|
|product_013799|       1|
|product_034593|       1|
+--------------+--------+
only showing top 20 rows



In [30]:
df50=spark.sql("""select /*+  SHUFFLE_HASH(s) */   product_name, count(*)  
            from products p join sales s on p.product_id = s.product_id
            group by 1 
            order by 2 asc 
            """)
print(df50.count())
df50.show()

1981
+--------------+--------+
|  product_name|count(1)|
+--------------+--------+
|product_043227|       1|
|product_074236|       1|
|product_015968|       1|
|product_053513|       1|
|product_038272|       1|
|product_042787|       1|
|product_044786|       1|
|product_088450|       1|
|product_006357|       1|
|product_081288|       1|
|product_045124|       1|
|product_020277|       1|
|product_064223|       1|
|product_031153|       1|
|product_016227|       1|
|product_085683|       1|
|product_050309|       1|
|product_099365|       1|
|product_034593|       1|
|product_046895|       1|
+--------------+--------+
only showing top 20 rows



In [31]:
df50=spark.sql("""select /*+  SHUFFLE_MERGE(s) */   product_name, count(*)  
            from products p join sales s on p.product_id = s.product_id
            group by 1 
            order by 2 asc 
            """)
print(df50.count())
df50.show()

1981
+--------------+--------+
|  product_name|count(1)|
+--------------+--------+
|product_006357|       1|
|product_015968|       1|
|product_038272|       1|
|product_042787|       1|
|product_043227|       1|
|product_044786|       1|
|product_053513|       1|
|product_074236|       1|
|product_081288|       1|
|product_088450|       1|
|product_016227|       1|
|product_020277|       1|
|product_031153|       1|
|product_045124|       1|
|product_050309|       1|
|product_064223|       1|
|product_085683|       1|
|product_007865|       1|
|product_013799|       1|
|product_034593|       1|
+--------------+--------+
only showing top 20 rows



In [32]:
####  Dataframe Joins
print(sales_table.join(broadcast(sellers_table), sales_table["seller_id"] == sellers_table["seller_id"], "inner").withColumn(
    "ratio", sales_table["num_pieces_sold"]/sellers_table["daily_target"]
).groupBy(sales_table["seller_id"]).agg(avg("ratio")).show())

+---------+--------------------+
|seller_id|          avg(ratio)|
+---------+--------------------+
|       29|4.644988939825174...|
|       26|2.319554387605397...|
|       65|2.931429678805068...|
|       54|2.389661293535090...|
|       19|2.858258703574406...|
|       22|3.708259605141521E-5|
|        7|1.555967948441298...|
|       77|2.981032682717429...|
|       34|3.477284846496053...|
|       50|2.870018587854760...|
|       94|4.674046551971184E-5|
|       57|3.405221429066354E-5|
|       32|8.634325709528986E-5|
|       43|2.882534893949249...|
|       84|4.044736739416897...|
|       31|3.006195752651825...|
|       98|6.342958088068233E-5|
|       39|5.703705240055282...|
|       25|3.625800692872154...|
|       95|2.924769017292419...|
+---------+--------------------+
only showing top 20 rows

None


In [33]:
from pyspark.sql import Row, Window
from pyspark.sql.types import IntegerType
import hashlib

#   Define the UDF function
def algo(order_id, bill_text):
    #   If number is even
    ret = bill_text.encode("utf-8")
    if int(order_id) % 2 == 0:
        #   Count number of 'A'
        cnt_A = bill_text.count("A")
        for _c in range(0, cnt_A):
            ret = hashlib.md5(ret).hexdigest().encode("utf-8")
        ret = ret.decode('utf-8')
    else:
        ret = hashlib.sha256(ret).hexdigest()
    return ret

#   Register the UDF function.
algo_udf = spark.udf.register("algo", algo)

sales_table = sales_table.withColumn("hashed_bill", algo_udf(col("order_id"), col("bill_raw_text")))

sales_table.show()


+--------+----------+---------+----------+---------------+--------------------+--------------------+
|order_id|product_id|seller_id|      date|num_pieces_sold|       bill_raw_text|         hashed_bill|
+--------+----------+---------+----------+---------------+--------------------+--------------------+
|       1|     57164|       69|2021-10-04|             20|tvuerrnvlBwhejtyz...|76c669d6d3e9c8ddd...|
|       2|     17466|        4|2021-10-03|             80|ezoexpsddxnahgkxr...|ezoexpsddxnahgkxr...|
|       3|     25747|       81|2021-10-08|             24|rrzavxtphkxhyiicu...|11d2ead66af66ec03...|
|       4|     53745|       59|2021-10-02|             92|jklcqmgjbfqxtsugt...|jklcqmgjbfqxtsugt...|
|       5|     31885|        5|2021-10-06|             10|azdujdzlhddjebywm...|4ac76798d905e2ba4...|
|       6|     36014|       52|2021-10-08|             71|jpjwwfckwolwphpkq...|63f8773c95665bb5e...|
|       7|      1709|       59|2021-10-05|              3|gcjrymmjlcmkhoyxq...|96a5d3cfb3d8

In [34]:
spark.stop()